# Libs

In [16]:
import os
import pandas as pd
import sys
# sys.path.append(os.path.join(os.getcwd(), 'src'))
# from parser_0 import parse_bamns_data
import requests
from tqdm import tqdm

# Giperparams

In [21]:
# url = 'https://box.contractor.ru/index.php/s/yycD4W6nC1ihe57/download?path=%2F04042023&files=mpsud-2023-04-04_07-32-29.bin' # test link
url = 'https://box.contractor.ru/index.php/s/yycD4W6nC1ihe57'

# path_to_csv_file_list = os.path.join('src', 'parser_bin', 'file_links.csv') # link-file df
path_to_csv_file_list = os.path.join('data', 'file_links.csv') # link-file df
path_to_csv = os.path.join('data', 'csv_data') # folder path
path_to_bin = os.path.join('data', 'bin_data') # folder path
list_of_cheching_data_types = ['_mpsud', '_mssdu', '_vibro', '_tkp'] # префиксы обработанных файлов

# Parse links from owncloud
Получаем ссылки на файлы в .csv, для их дальнейшего скачивания или просмотра их наличия.
Доп комментарии:
Страница динамически обновляется, больше тысячи файлов для анализа. Платформа не предполагает получение информации таким образом, поэтому имитируем действия человека, т.е. можно было бы использовать более эффективные алгоритмы... но оптимировать или исправлсять ошибки их нет желания
При слабом соединении увеличить time.sleep
В фоновый режим парсинг лучше не ставить

In [38]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import re

root = '/html/body/div[2]/div/div/div[1]/div[1]/div/a'

def current_state(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup
def scroll(driver):
    ### scroll main page
    state = current_state(driver)  # запоминаем текущее состояние
    # неебиечский код для скроллера
    scroll_origin = ScrollOrigin.from_element(driver.find_element(By.XPATH, '//*[@id="fileList"]'))
    while True:
        ActionChains(driver) \
            .scroll_from_origin(scroll_origin, 0, 1000) \
            .perform()
        curr_state = current_state(driver)
        if curr_state == state:  # если состояние не меняется, то выходим из цикла и заканчиваем скролл
            break
        state = curr_state

def simple_scroll(driver):
    scroll_origin = ScrollOrigin.from_element(driver.find_element(By.XPATH, '//*[@id="fileList"]'))
    ActionChains(driver).scroll_from_origin(scroll_origin, 0, 200).perform()
    time.sleep(0.2) # на всякий случай


def parse_bin_links(links_list):
    update = False
    for link in re.findall(r'href="/index.php/s/yycD4W6nC1ihe57([^"]+).bin', driver.page_source):
        full_link = (url + link + '.bin').replace('amp;', '')
        if full_link not in links_list:
            links_list.append(full_link)
            update = True
    return update

def update_links_list():
    # simple_scroll(driver) #  может вызывать ошибки
    update = True
    while update:
        update = parse_bin_links(links_list)
        simple_scroll(driver)

root = '/html/body/div[2]/div/div/div[1]/div[1]/div/a'
data = {}
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)

links_list = []
# Открываем корневой каталог
block = driver.find_element(By.XPATH, '//*[@id="fileList"]')  # основная директория
number_of_folder = 1
#### блок обработки папок
for number_of_folder, x in enumerate([x.split('\n')[0] for x in block.text.split('ago\n')]):
    if len(x) == 8:
        # dir_list.append(x)
        print(f'check {x} folder')
        xpath = f'//*[@id="fileList"]/tr[{number_of_folder+1}]/td[1]/a'  # путь до папки
        # делаем элемент кликабельным?
        # element = WebDriverWait(driver, 1000).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        # time.sleep(1)
        # открываем каталог
        driver.find_element(By.XPATH, xpath).click()
        time.sleep(2)
        # парсим ссылки регулярными выражениями и добавляем их в список
        update_links_list()

        # print(f'going back to root')
        print(f'links size {len(links_list)}')
        driver.find_element(By.XPATH, root).click()
        time.sleep(2)

print('check root')
update_links_list()

print(len(links_list))
print(links_list[-1]) # для проверки корректности ссылки

df = pd.DataFrame({'link': links_list, 'file_name': [x.split('files=')[1] for x in links_list]})
df.sort_values(by = ['file_name'], ascending=False, inplace=True)
df.to_csv(path_to_csv_file_list,sep = '\t')

check 04042023 folder
links size 4
check 06072023 folder
links size 237
check 07042023 folder
links size 240
check 07052023 folder
links size 423
check 07062023 folder
links size 501
check 14042023 folder
links size 726
check 16022023 folder
links size 893
check 18052023 folder
links size 962
check 20042023 folder
links size 964
check 21062023 folder
links size 1128
check 24032023 folder
links size 1700
check 27042023 folder
links size 1938
check 30052023 folder
links size 2038
check root
2038
https://box.contractor.ru/index.php/s/yycD4W6nC1ihe57/download?path=%2F30052023&files=mpsud-2023-05-30_13-52-38.bin


## Check non unique files

In [40]:
df.sort_values(by = ['file_name'], ascending=False, inplace=True)

# with open('temp.txt', 'w') as file:
for i in df.file_name.unique():
    temp_df = df[df.file_name == i]
    if temp_df.shape[0] > 1:
        print(i)
        # file.write('\n')
        # print(list(temp_df.link))

# Downdoad file by url
Здесть мы проверяем наличие обработанных файлов по суффиксам даты файла и list_of_cheching_data_types. Если в системе нет этих обработанных таблиц, то скачиваем и парсим бинарник в формат .csv.
Есть ограничение на скорость загрузки одного файла, поэтому ставим парралельную загрузку (в данном случае на другом ядре, можете придумать получше)))

In [3]:
def download(url: str, fname: str, chunk_size=2**28):
    # download, bar and buffer function
    resp = requests.get(url, stream=True, verify=False)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
        desc=fname,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=chunk_size,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)
            
def run_process():
    # download, check and precess function
    
    contractor_data_pathes = pd.read_csv(path_to_csv_file_list) # файл с df от сайта сколтех с ссылками и названиями файлов
    contractor_data_pathes.sort_values(by = ['file_name'], ascending=False, inplace=True)
    
    csv_list = [x for x in os.listdir(path_to_csv) if '.csv' in x] # список уже загруженных файлов
    csv_list.sort(reverse = True)  # это не обязательно

    for row in contractor_data_pathes.iterrows(): # check каждую строчку из файла со ссылками
        _, row = row # убираем индексы
        
        bin_file_path = os.path.join(path_to_bin, row.file_name) # объявляем файл бинарника (соблюдаем приличия для его названия
        list_of_cheking_files = [row.file_name[6:-4] + x + '.csv' for x in list_of_cheching_data_types] # лист для проверки обработанных файлов
        
        # проверяем наличие обработанных файлов по бинарнику и само наличие бинарника (для парралельной загрузки). Если нет, то запусаем в обработку
        if (sum([x in csv_list for x in list_of_cheking_files])/len(list_of_cheking_files) != 1.0) and (not os.path.exists(bin_file_path)):
            # print(row.link)
            # скачиваем файл
            download(row.link, bin_file_path)
            # обрабатываем файл
            
            # parse_bamns_data(path_to_bin, path_to_csv)
            
            ###### ОДИН ФАЙЛ!!!
            parse_bamns_data_file(bin_file_path, path_to_csv)
            # удаляем файл
            os.remove(bin_file_path)
        else:
            # пропуск, если файл уже есть
            # print (f'{row.file_name} is processed')
            pass
            
# run_process()

## Parallel download
Сколтех (или чтото ещё) ставит ограничение на скорость загрузки одного файла. Обходим это условие одновременным скачиванием нескольких файлов (run_threads(4) - четыре файла)

In [ ]:
import threading
import sys
import time


def thread_job(number):
    print('Hello {}'.format(number))
    # инициализируем основной процесс
    run_process()
    sys.stdout.flush() # хз


def run_threads(count):
    threads = [
        threading.Thread(target=thread_job, args=(i,))
        for i in range(0, count)
    ]
    for thread in threads:
        thread.start()
        time.sleep(5) # тормозим и ждем инициализации бинарного файла в файловой системе, для работы условного оператора из основной функции
    for thread in threads:
        thread.join()  # дожидаемся исполнения всех потоков

run_threads(4)
print('finish')

Hello 0


/home/VNIIZHT.LAN/nikolaev.mikhail/anaconda3/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'box.contractor.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
data/bin_data/mpsud-2023-07-06_22-59-06.bin:   0%|                                          | 0.00/33.2k [00:00<?, ?iB/s]

Hello 1


/home/VNIIZHT.LAN/nikolaev.mikhail/anaconda3/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'box.contractor.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(

data/bin_data/mpsud-2023-07-06_21-59-06.bin:   0%|                                          | 0.00/33.2k [00:00<?, ?iB/s]

Hello 2


...


data/bin_data/mpsud-2023-07-02_05-44-49.bin:  15%|████▉                            | 5.00k/33.2k [05:44<32:31, 3.88MiB/s]

csv "data/csv_data/2023-07-02_06-44-49_mssdu.csv" create
csv "data/csv_data/2023-07-02_06-44-49_vibro.csv" create



data/bin_data/mpsud-2023-07-02_05-44-49.bin:  15%|████▉                            | 5.00k/33.2k [06:00<32:31, 3.88MiB/s]

csv "data/csv_data/2023-07-02_06-44-49_tkp.csv" create
Execution time for data/bin_data/mpsud-2023-07-02_06-44-49.bin: 48.299803733825684 seconds


/home/VNIIZHT.LAN/nikolaev.mikhail/anaconda3/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'box.contractor.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
data/bin_data/mpsud-2023-06-30_22-20-29.bin:   0%|                                          | 0.00/30.8k [00:00<?, ?iB/s]

data/bin_data/mpsud-2023-07-04_10-06-52.bin:  87%|████████████████████████████▊    | 29.0k/33.2k [33:56<04:52, 3.83MiB/s]

data/bin_data/mpsud-2023-07-04_10-06-52.bin:  87%|████████████████████████████▊    | 29.0k/33.2k [34:11<04:52, 3.83MiB/s]
data/bin_data/mpsud-2023-07-02_05-44-49.bin:  18%|█████▉                           | 6.00k/33.2k [06:54<31:37, 3.85MiB/s]
data/bin_data/mpsud-2023-06-30_22-20-29.bin:   3%|█                                | 1.00k/30.8k [01:14<37:13, 3.58MiB/s]

data/bin_data/mpsud-2023-06-30_22-20-29.